In [35]:
# Each line after the first is either broad, medium, or narrow.
# Also sum for each industry.
# For the first attempt, just ignore the non-national data.
# Descending order of size: Major, Minor, Broad, Detail.
# For NAICS values, all are either two-digit or six digit, right-padded with 0s.
import csv
import re
with open("all_data_M_2015.csv",'r') as f:
    c=csv.reader(f)
    L=list(c)
print(L[0])

['area', 'area_title', 'area_type', 'naics', 'naics_title', 'own_code', 'occ code', 'occ title', 'group', 'tot_emp', 'emp_prse', 'jobs_1000', 'loc_quotient', 'pct_total', 'h_mean', ' a_mean ', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', ' a_pct10 ', ' a_pct25 ', ' a_median ', ' a_pct75 ', ' a_pct90 ', 'annual', 'hourly', '']


In [9]:
d=dict()
def get(count):
    return [j[count] for j in L[1:]]
d['naics']= get(3)     #naics
d['occ_code']= get(6)  #occ code, adjusted name here for consistency
d['group']= get(8)     #Group
d['tot_emp']= get(9)     #Total employment
d['h_mean']=get(14)    #Mean Hourly Pay
d['a_mean']=get(15)    #Mean Annual Pay, adjusted the name here to remove extraneous spaces
d["h_pay_dist"]=get(slice(17,22)) #10th, 25th, 50th, 75th, and 90th percentile hourly pay.
d["a_pay_dist"]=get(slice(22,27)) #10th, 25th, 50th, 75th, and 90th percentile annual pay.
all_jobs=[[d['naics'][n],d['occ_code'][n],d['group'][n],d['tot_emp'][n],d['h_mean'][n],d['a_mean'][n],
          d["h_pay_dist"][n],d["h_pay_dist"][n]] for n in range(len(d['naics']))]
d.keys()

dict_keys(['group', 'naics', 'occ_code', 'tot_emp', 'h_mean', 'h_pay_dist', 'a_mean', 'a_pay_dist'])

In [39]:
pairs=[" ".join([a[3], a[4]]) for a in L[1:]]
set(pairs)


{'000000 Cross-industry',
 '000001 Cross-industry, private ownership only',
 '11 Agriculture, Forestry, Fishing and Hunting',
 '113000 Forestry and Logging',
 '113300 Logging',
 '115000 Support Activities for Agriculture and Forestry',
 '115100 Support Activities for Crop Production',
 '115200 Support Activities for Animal Production',
 '21 Mining, Quarrying, and Oil and Gas Extraction',
 '211000 Oil and Gas Extraction',
 '211100 Oil and Gas Extraction',
 '212000 Mining (except Oil and Gas)',
 '212100 Coal Mining',
 '212200 Metal Ore Mining',
 '212300 Nonmetallic Mineral Mining and Quarrying',
 '213000 Support Activities for Mining',
 '213100 Support Activities for Mining',
 '22 Utilities',
 '221000 Utilities',
 '221100 Electric Power Generation, Transmission and Distribution',
 '221111 Hydroelectric Power Generation',
 '221112 Fossil Fuel Electric Power Generation',
 '221113 Nuclear Electric Power Generation',
 '221114 Solar Electric Power Generation',
 '221115 Wind Electric Power Gen

In [10]:
def select(func,universe_list=all_jobs):
    final=list()
    for i in universe_list:
        if func(i):
            final.append(i)
    return final

def match(index,value):
    return lambda x:(x[index]==value)

def access_job(occ_code,universe_dict=d,universe_list=all_jobs):
    code_list=universe_dict['occ_code']
    holder=[]
    for i,q in enumerate(code_list):
        if q==occ_code:
            holder.append(universe_list[i])
    return holder
#access_job("13-2000")

In [11]:
def get_percent(job_num,naics_codes): #job_num should be a string, naics_code a list of relevant naics codes.
    # Also works fine if you just use a single string.
    job_rows=access_job(str(job_num))
    denominator=0
    for row in job_rows:
        if row[0] in naics_codes:
            numerator=int(re.sub(",","",row[3]))
        try:
            denominator+=int(re.sub(",","",row[3]))
        except ValueError:
            #print(row[3])
            pass
    return numerator/denominator
get_percent("13-2000","541200") # Financial Specialists and Accounting, Tax Preparation, Bookkeeping, and Payroll Services
def get_highest(job_num):
    industry_list=[a for a in sorted(set(d['naics']))[2:] if len(a)==6 and a[3]=='0']
    highest,denominator=0,0
    all_job_rows=access_job(str(job_num)) #Gets all rows that contain the job.
    rows=[a for a in all_job_rows if a[0] in industry_list] #Narrows it to exclude unwanted categories.
    for row in rows:
        try:
            num=int(re.sub(",","",row[3]))
        except ValueError:
            print(row)
            continue
        if num>highest:
            highest=num
            highest_row=row
        denominator+=int(re.sub(",","",row[3]))
    return highest_row,highest/denominator

In [12]:
job_list=[a for a in sorted(set(d['occ_code'])) if a[-1]!='0']
x=[[get_highest(job),job] for job in job_list]

['523000', '11-1011', 'detail ', '**', '116.63', '242,590', ['58.15', '89.79', '#', '#', '#'], ['58.15', '89.79', '#', '#', '#']]
['332000', '11-2011', 'detail ', '**', '45.44', '94,520', ['32.82', '39.52', '43.19', '46.86', '55.87'], ['32.82', '39.52', '43.19', '46.86', '55.87']]
['532000', '11-2011', 'detail ', '**', '45.20', '94,010', ['22.46', '31.80', '41.76', '57.55', '68.34'], ['22.46', '31.80', '41.76', '57.55', '68.34']]
['533000', '11-2011', 'detail ', '**', '53.50', '111,270', ['22.39', '30.53', '36.59', '70.34', '#'], ['22.39', '30.53', '36.59', '70.34', '#']]
['451000', '11-2021', 'detail ', '**', '66.68', '138,700', ['20.34', '22.45', '50.36', '#', '#'], ['20.34', '22.45', '50.36', '#', '#']]
['446000', '11-2022', 'detail ', '**', '46.15', '96,000', ['21.36', '27.41', '38.89', '57.46', '80.02'], ['21.36', '27.41', '38.89', '57.46', '80.02']]
['533000', '11-2031', 'detail ', '**', '49.31', '102,570', ['30.80', '37.10', '46.89', '64.09', '74.10'], ['30.80', '37.10', '46.89'

In [30]:
get_highest("15-1100")

(['541000',
  '15-1100',
  'minor',
  '1,403,690',
  '43.20',
  '89,860',
  ['20.73', '29.37', '40.90', '55.14', '69.90'],
  ['20.73', '29.37', '40.90', '55.14', '69.90']],
 0.3104389346319801)

In [26]:
import matplotlib
from matplotlib import style
style.use('ggplot')
import pandas as pd
import numpy as np
concentration_percentages=[a[0][1] for a in x]
df=pd.DataFrame(concentration_percentages)
S=pd.Series(concentration_percentages)

In [27]:
S.describe()

count    820.000000
mean       0.457181
std        0.230366
min        0.087248
25%        0.294113
50%        0.392527
75%        0.575624
max        1.000000
dtype: float64

In [28]:
S.plot()

[0.1298273893478407,
 0.09342833848557937,
 0.5,
 0.36054648450516497,
 0.21367818950509823,
 0.09165819759936276,
 0.14511388684790597,
 0.1481073233681041,
 0.26762546515136276,
 0.15849644055978077,
 0.10887214531861457,
 0.1510910815939279,
 0.11976808831705187,
 0.20555846879916098,
 0.12438551099611901,
 0.14088181581007045,
 0.5061224489795918,
 0.37745486221095975,
 0.5582355856600033,
 0.3477430894797757,
 0.39600753739253325,
 0.29262910107400325,
 0.32263136538851694,
 0.7190871940860462,
 1.0,
 0.3673835125448029,
 0.921370401572592,
 0.24700562209728671,
 0.3090909090909091,
 0.5,
 0.7291190242840031,
 0.31026814911706996,
 0.38960313024035775,
 0.23614340358508962,
 0.9200930954228084,
 0.3587982832618026,
 0.18886910062333037,
 0.15719733340649092,
 0.5857575479489768,
 0.9564068692206077,
 0.3099147094229439,
 0.3828495192528868,
 0.16745647890305665,
 0.9338842975206612,
 0.7779879418370966,
 0.17412445374530683,
 0.30613175587637464,
 0.14568957138691807,
 0.295347744